Mumford-Shah image segmentation
===============================

This code provides a CPU (slow\*) implementation of an approximation to Mumford-Shah image segmentation. 

Based on:
 * "A first-order primal-dual algorithm for convex problems with applications to imaging" 
Chambolle, Antonin and Pock, Thomas (2011)
Journal of Mathematical Imaging and Vision. 40(1)

Intersection of convex set projection method based on 
 * "A cyclic projection algorithm via duality"
Gaffke, Norbert and Mathar, Rudolf (1989)
Metrika. 36(1)

Unit simplex projection based on 
 * "Projection onto the probability simplex : An efficient algorithm with a
 simple proof and an application"
Wang, Weiran and Miguel, A (2013)
arXiv:1309.1541v1

\* GPU implementation coming... 

![title](https://github.com/benlansdell/segmentation/blob/master/butterfly.png)


What are we doing?
---------------------

The basic idea is to fix a set of $k$ colors we are going to segment the image with. We want to do so in a way that avoids high spatial variability while maintaining sharp transitions between regions of very different color. The solution is to use a total-variation (TV) penalty. Let $\{c_i\}^k$ be our set set of colors and let $g(\mathbf{x})$ be our image. Let $\mathbf{f}$ be the vector of distance between the image and each of our colors:
$$
f_l = \lambda \int |g(x)-c_i|^2\,d\mathbf{x}\qquad l = 1,\dots, k
$$

Let $\mathbf{u}(x)$ be our vector that weights contributions from each color $c_i$ to produce a final pixel intensity at $x$:
$$
I(x) = u(x)^T\mathbf{c}.
$$
Thus $\mathbf{u}$ selects our color. We want the vector $\mathbf{u}$ to be _sparsely_ varying as a function of $x$.

We set this up as the following optimization problem
$$
\min_{u\in U} \frac{1}{2}\sum_{l=1}^k \int_\Omega |Du_l| + \sum_{l=1}^k\int_\Omega u_lf_l\,dx
$$
subject to particular constraints described more below. 

Why TV minimization?
-----------------------

Here $\int_\Omega|Du|$ denotes the total variation of the function $u$, with $Du$ being the _distributional derivative_ of $u$. For sufficiently smooth functions, $u\in W^{1,1}(\Omega)$, it reduces to:
$$
\int_\Omega |Du| = \int_\Omega|\nabla u|\,dx.
$$

There is a nice relation between the total variation regularization and perimeter/partition minimization. This is allowed by the fact that
$$
\int_\Omega |Du| = \int_{-\infty}^{\infty}\text{Per}(\{u>s\},\Omega)\,ds,
$$
that is, that the total variation of $u$ is equivalent to the sum of the surfaces of its level sets. This means minimizing TV is equivalent to minimizing the length of boundaries of contiguous regions of color. 

Relation to image segmentation
----------------------------------

Coming back to our image segmentation problem. Let $R_l$ denote a partition of the domain $\Omega$. Each partition has a color $c_i$. The piece-wise constant, Mumford-Shah problem is then
$$
\min_{(R_l)_{l=1}^k, (c_l)_{l=1}^k}\frac{1}{2}\sum_{l=1}^k \text{Per}(R_l;\Omega) + \frac{\lambda}{2}\sum_{l=1}^k \int_{R_l}|g(x)-c_l|^2\,dx.
$$
The original specification treats the colors $c_l$ as unknown. This is related to a model from statistical physics known as the [Potts model](https://en.wikipedia.org/wiki/Potts_model). However this problem can be shown to be NP-hard. A convex problem can be obtained by treating $c_l$ are known. (An easy way to choose sensible colors is to choose a $k$, and perform k-means clustering to pick the $c_l$s.)

We thus relate the minimal partition problem above to a type of vector total variation minimization:
$$
\min_{(u_l)_{l=1}^k}J(u) + \sum_{l=1}^k \int_\Omega u_lf_l \,dx,\quad u_l(x)\ge 0, \sum_{l=1}^k u_l(x) = 1,\forall x\in \Omega
$$
Where different choices of energy $J(u)$ are possible. The simplest is to choose
$$
J_1(u) = \frac{1}{2}\sum_{k=1}^l \int_\Omega |Du_l|,
$$
and a more principled choice is
$$
J_2(u) = \int_\Omega \Psi(Du),
$$
where
$$
\Psi(Du) = \sup_q\{\langle p,q:|q_l-q_m|\le 1,1\le l<m\le k \rangle \}.
$$

In this implementation we use the simpler functional $J_1(u)$. 

The primal-dual implementation we solve is then:

$$
\min_{u=(u_l)_{l=1}^k}\max_{p=(p_l)_{l=1}^k}\left(\sum_{l=1}^k\langle \nabla u_l, p_l\rangle + \langle u_l,f_l\rangle \right)+\delta_U(u) - \delta_P(p).
$$
Where $U$ is the pixel-wise unit simplex:
$$
U=\left\{u\in X^k:(u_l)_{i,j}\ge 0, \sum_{l=1}^k (u_l)_{i,j} = 1\right\}
$$
and $P$ is the set
$$
P = \left\{ p\in Y^k,\|p_l\|_\infty \le \frac{1}{2} \right\}
$$

How do we solve this?
------------------------



The Chambolle algorithm solves problems with the following setup. Let $X$ and $Y$ be two finite dimensional real vector spaces with inner product/norms, $\langle \cdot,\cdot\rangle_{X/Y}$ and $\|\cdot\|_{X/Y} = \langle \cdot, \cdot \rangle^{1/2}$. Whether the inner product applies to $X$ or $Y$ should be able to be inferred from context, so will not be denoted. Let $K:X\to Y$ be a linear operator having adjoint $K^*$. The generic problem that is solved is:
$$
\min_{x\in X} \max_{y\in Y} \langle Kx, y\rangle + G(x) - F^*(y)
$$
where $G:X\to[0,\infty]$ is a lower semi-continuous convex function, and $F^*:Y\to[0,\infty]$ is also lower semi-continuous and convex. In primal form this is equivalent to solving:
$$\min_{x\in X} F(Kx) + G(x).$$

The solution $(\hat{x},\hat{y})$ satisfies:
$$\begin{align*}
K\hat{x}\in \partial F^*(\hat{y})\\
-(K^*\hat{x})\in \partial G(\hat{x})
\end{align*}$$

We define the resolvent, or proximal, function of a convex function $F$ as:
$$\begin{align}
x&= (I+\tau \partial F)^{-1}(y)\\
&= \text{argmin}_{x}\left\{\frac{\|x-y\|^2}{2\tau}+F(x)\right\}\\
&= \text{prox}_F(y;\tau)
\end{align}$$
If $F$ is an indicator function then $\text{\prox}_F(y;\tau)$ is just a Euclidean projection onto the convex set indicated by $F$. (Note the resolvent is a generalization of proximal operators for maximal monotone operators, but this distinction won't concern us here).

We present the following algorithm, known as the Chambolle algorithm, to solve problems of the above form. Recall,  generically, the proximal point algorithm takes the following form:
$$
x_{n+1} = (I+\lambda_n \partial F)^{-1})(x_n)
$$
for convex function $F$. This can be seen as solving the root finding equation
$$
0\in \partial F(x)
$$
through a sort of backward Euler (implicit), gradient descent iteration. The Chambolle algorithm applies the proximal point method to primal-dual system. The above is generalized to a system of equations:
$$
0\in T(x^{n+1})+M^{-1}(x^{n+1}-x^n)
$$
for 
$$
T = \left(\begin{array}{c}\partial G(x^{n+1}) +K^*y^{n+1}\\\partial F^*(y^{n+1}) -K^*x^{n+1}\end{array}\right)\quad\text{and}\quad M=\left(\begin{array}{cc}\mathcal{T}^{-1}&-K^*\\-K&\Sigma^{-1}\end{array}\right).
$$
The combination of these two equations gives the following algorithm:

1. Init: $\tau,\sigma > 0$ with $\tau\sigma L^2 < 1$, $\theta \in [0,1], (x^0,y^0)\in X\times Y, \bar{x}^0 = x^0$
2. Repeat until convergence:
   $$
   \begin{align}
   y^{n+1} &= \text{prox}_{F^*}(y^n + \sigma K\bar{x}^n; \sigma)\\
   x^{n+1} &= \text{prox}_{G}(x^n - \tau K^* y^n; \tau)\\
   \bar{x}^{n+1} &= x^{n+1} + \theta(x^{n+1}-x^n)
   \end{align}
   $$
   
Thus application of the algorithm to different problems merely involves different choices of the proximal operators and operator $K$. Note that we can slightly generalize the algorithm to deal with problems of the form:
$$
\min_{x\in X} \max_{y\in Y} \langle Kx, y\rangle + \langle g , x \rangle - \langle h , y \rangle + G(x) - F^*(y)
$$
with
1. Init: $\tau,\sigma > 0$ with $\tau\sigma L^2 < 1$, $\theta \in [0,1], (x^0,y^0)\in X\times Y, \bar{x}^0 = x^0$
2. Repeat until convergence:
   $$
   \begin{align}
   y^{n+1} &= \text{prox}_{F^*}(y^n + \sigma (K\bar{x}^n-h); \sigma)\\
   x^{n+1} &= \text{prox}_{G}(x^n - \tau (K^* y^n  - g); \tau)\\
   \bar{x}^{n+1} &= x^{n+1} + \theta(x^{n+1}-x^n)
   \end{align}
   $$

Applications to imaging
--------------------------

If we have an image of dimensions $M\times N$ then left $X = \mathbb{R}^{M\times n}$ and let the following be given by
$$\begin{align}
\langle u,v\rangle &= \sum_{ij}u_{ij}v_{ij}\\
\nabla:&X\to Y = \text{forward difference grad operator}\\
(\nabla u)_{ij} &= \left(\begin{array}{c}(\nabla u)_{ij}^1\\(\nabla u)_{ij}^2\end{array}\right)\\
\end{align}$$
with the adjoint operator being 
$$
-div:Y\to X = \text{backward difference div operator}
$$
Note that using forward and backward differencing matters here. $L$ is the Lipschitz constant, and satisfies:
$$
L^2 = \|\nabla\|^2 = \|\text{div}\|^2 = 8
$$

With these preliminaries we can now apply the Chambolle algorithm to the image segmentation problem above. We simply identify $F^*(p) = \delta_P(p)$ and $G(u) = \delta_U(u)$ and compute the resolvent operators. In both cases they become pixel-wise operations that can be easily accelerated on a GPU.

Computing the resolvent operators
--------------------------------------

We have:
$$
\begin{align}
\text{prox}_G(\tilde{u};\tau) &= \text{argmin}_u\left\{\frac{\|u-\tilde{u}\|^2}{2\tau} + \delta_U(u)\right\}\\
&= \text{per-pixel projection onto unit simplex}
\end{align}
$$
and
$$
\begin{align}
\text{prox}_{F^*}(\tilde{p};\sigma) &= \text{argmin}_p\left\{\frac{\|p-\tilde{p}\|^2}{2\sigma} + \delta_P(p)\right\}\\
&= \text{per-pixel projection onto unit balls}
\end{align}
$$
where
$$
p_{i,j}^l = \frac{\tilde{p}_{ij}^l}{\max(1,2|\tilde{p}_{ij}^l|)}
$$
and
$$
|p_{ij}^l| = \left[(p_{ij}^1)^2 + (p_{ij}^2)^2\right]^{1/2}
$$

Thus we can solve the image segmentation problem with the Chambolle algorithm.